# Strategy Comparison

Compare performance metrics across multiple strategies.

**Version**: v1.11.0  
**Architecture**: Modular (lib/metrics/, lib/paths/)

## Configuration

Set the strategy names to compare below.

**Note**: This notebook uses the v1.11.0 modular architecture. All imports use canonical paths from `lib/` packages.


In [ ]:
# Configuration
strategy_names = ['spy_sma_cross']  # Add more strategies to compare


In [ ]:
# Setup
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Standard library imports
import json

# Third-party imports
import pandas as pd
import numpy as np
from IPython.display import Image, display

# Local imports - lib modules (v1.11.0 modular architecture)
# Comparison functions provide multi-strategy analysis
from lib.paths import get_project_root, get_results_dir
from lib.metrics import compare_strategies


In [ ]:
# List available strategies with results (optional)
results_base = get_results_dir()

if results_base.exists():
    available_strategies = []
    for strategy_dir in results_base.iterdir():
        if strategy_dir.is_dir() and (strategy_dir / 'latest').exists():
            available_strategies.append(strategy_dir.name)
    
    if available_strategies:
        print("Available strategies with results:")
        for i, name in enumerate(sorted(available_strategies), 1):
            print(f"  {i}. {name}")
        print(f"\nTotal: {len(available_strategies)} strategies")
    else:
        print("⚠ No strategies with results found")
        print(f"  Run backtests first: python scripts/run_backtest.py --strategy <name>")
else:
    print(f"⚠ Results directory not found: {results_base}")


In [ ]:
# Load results for multiple strategies
results_data = {}
results_base = get_results_dir()

print("Loading results for strategies:")
for strategy_name in strategy_names:
    strategy_dir = results_base / strategy_name / 'latest'
    
    if strategy_dir.is_symlink():
        strategy_dir = strategy_dir.resolve()
    
    if not strategy_dir.exists():
        print(f"  ✗ {strategy_name}: Results not found at {strategy_dir}")
        continue
    
    try:
        # Load metrics
        metrics_file = strategy_dir / 'metrics.json'
        if not metrics_file.exists():
            print(f"  ✗ {strategy_name}: metrics.json not found")
            continue
        
        with open(metrics_file) as f:
            metrics = json.load(f)
        
        # Load returns
        returns_file = strategy_dir / 'returns.csv'
        if returns_file.exists():
            returns = pd.read_csv(returns_file, index_col=0, parse_dates=True)
        else:
            returns = None
        
        results_data[strategy_name] = {
            'metrics': metrics,
            'returns': returns,
            'path': strategy_dir
        }
        
        print(f"  ✓ {strategy_name}: Loaded from {strategy_dir.name}")
        
    except Exception as e:
        print(f"  ✗ {strategy_name}: Error loading - {e}")
        continue

print(f"\n✓ Loaded {len(results_data)} strategies")


In [ ]:
# Bundle and Calendar Information for Strategies
from lib.bundles import get_bundle_symbols
from lib.calendars import get_calendar_for_asset_class
from lib.config import load_strategy_params

print("\nStrategy Information:")
for strategy_name, data in results_data.items():
    # Try to load strategy params to get asset class
    try:
        params = load_strategy_params(strategy_name)
        asset_class = params.get('strategy', {}).get('asset_class', 'equities')
        calendar = get_calendar_for_asset_class(asset_class)
        
        print(f"\n  {strategy_name}:")
        print(f"    Asset Class: {asset_class}")
        print(f"    Calendar: {calendar}")
        
    except Exception as e:
        print(f"  {strategy_name}: Could not load info ({e})")

# Compare strategies
if len(results_data) < 2:
    print("\n⚠ Need at least 2 strategies to compare")
    print(f"  Loaded: {len(results_data)} strategies")
else:
    try:
        # Extract returns for comparison
        returns_dict = {}
        for name, data in results_data.items():
            if data['returns'] is not None and 'returns' in data['returns'].columns:
                returns_dict[name] = data['returns']['returns']
        
        if len(returns_dict) >= 2:
            # Use library function for comparison
            comparison = compare_strategies(returns_dict)
            
            print("\n" + "=" * 60)
            print("STRATEGY COMPARISON")
            print("=" * 60)
            print(comparison.to_string())
            print("=" * 60)
            
        else:
            print("\n⚠ Insufficient returns data for comparison")
            
    except Exception as e:
        print(f"✗ Comparison error: {e}")
        raise

In [ ]:
# Display detailed comparison
if len(results_data) >= 2:
    print("\n" + "=" * 60)
    print("DETAILED METRICS COMPARISON")
    print("=" * 60)
    
    # Create comparison DataFrame
    comparison_data = []
    for name, data in results_data.items():
        metrics = data['metrics']
        comparison_data.append({
            'Strategy': name,
            'Total Return': metrics.get('total_return', 0),
            'Annual Return': metrics.get('annual_return', 0),
            'Sharpe Ratio': metrics.get('sharpe', 0),
            'Sortino Ratio': metrics.get('sortino', 0),
            'Max Drawdown': metrics.get('max_drawdown', 0),
            'Calmar Ratio': metrics.get('calmar', 0),
            'Volatility': metrics.get('annual_volatility', 0),
            'Trade Count': metrics.get('trade_count', 0),
            'Win Rate': metrics.get('win_rate', 0)
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df.set_index('Strategy', inplace=True)
    
    # Format percentages
    pct_cols = ['Total Return', 'Annual Return', 'Max Drawdown', 'Volatility', 'Win Rate']
    for col in pct_cols:
        if col in comparison_df.columns:
            comparison_df[col] = comparison_df[col].apply(lambda x: f"{x:.2%}" if pd.notna(x) else "N/A")
    
    print(comparison_df.to_string())
    print("=" * 60)
    
    # Identify best strategy for each metric
    numeric_df = pd.DataFrame(comparison_data)
    numeric_df.set_index('Strategy', inplace=True)
    
    print("\nBest Strategy by Metric:")
    metrics_to_compare = {
        'Total Return': 'max',
        'Annual Return': 'max',
        'Sharpe Ratio': 'max',
        'Sortino Ratio': 'max',
        'Calmar Ratio': 'max',
        'Max Drawdown': 'min',  # Lower is better
        'Volatility': 'min'     # Lower is better
    }
    
    for metric, direction in metrics_to_compare.items():
        if metric in numeric_df.columns:
            if direction == 'max':
                best = numeric_df[metric].idxmax()
                value = numeric_df[metric].max()
            else:
                best = numeric_df[metric].idxmin()
                value = numeric_df[metric].min()
            
            if pd.notna(value):
                print(f"  {metric}: {best} ({value:.3f})")

In [ ]:
# Visualize comparison
if len(results_data) >= 2:
    try:
        import matplotlib.pyplot as plt
        
        # Extract returns for plotting
        returns_dict = {}
        for name, data in results_data.items():
            if data['returns'] is not None and 'returns' in data['returns'].columns:
                returns_dict[name] = data['returns']['returns']
        
        if len(returns_dict) >= 2:
            # Plot cumulative returns
            fig, ax = plt.subplots(figsize=(12, 6))
            
            for name, returns in returns_dict.items():
                cumulative = (1 + returns).cumprod()
                ax.plot(cumulative.index, cumulative.values, label=name, linewidth=2)
            
            ax.set_title('Strategy Comparison - Cumulative Returns', fontsize=14, fontweight='bold')
            ax.set_xlabel('Date')
            ax.set_ylabel('Cumulative Return')
            ax.legend()
            ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
    except ImportError:
        print("⚠ Matplotlib not available for visualization")
    except Exception as e:
        print(f"⚠ Visualization error: {e}")